# Calculate FCM Statistics
*Written by Viviane Clay*
## Preparation
### Import Dependencies

In [4]:
import os
import numpy as np;
import matplotlib.pyplot as plt
from scipy import misc
import matplotlib as mpl
from itertools import combinations 
import pandas as pd
from sklearn import svm

### Load Data
Get activations using the 'GetNetworkActivations.ipynb' script. Adapt paths as nescessary.

In [21]:
path = './Results/TowerTraining/BalancedTestSet/'
figurePath = './Results/TowerTraining/Figures/AgentRewardComparisonsAdaTH/ActivationPatternsNormx2No-01-FlatC-50-50Test/XX'

encExt = np.load(path + 'encExt.npy')
encExtInt = np.load(path + 'encExtInt.npy')
encInt = np.load(path + 'encInt.npy')
encAE = np.load(path + 'encAE.npy')
encC = np.load(path + 'encC.npy')
ClassOut = np.load(path + 'outClass.npy')
y = np.load(path + 'labels.npy')
obs = np.load(path + 'obs.npy')
# generate random activations
randAct = np.random.randint(0,2,encC.shape)

### Apply Adaptive Threshold to Activations

In [22]:
c1ActTH = np.mean(encInt, axis = 0)
c2ActTH = np.mean(encExt, axis = 0)
c3ActTH = np.mean(encExtInt, axis = 0)
AEActTH = np.mean(encAE, axis = 0)
CActTH = np.mean(encC, axis = 0)

encInt = np.array(np.abs(encInt) > np.abs(c1ActTH),dtype=int)
encExt = np.array(np.abs(encExt) > np.abs(c2ActTH),dtype=int)
encExtInt = np.array(np.abs(encExtInt) > np.abs(c3ActTH),dtype=int)
encAE = np.array(np.abs(encAE) > np.abs(AEActTH),dtype=int)
encC = np.array(np.abs(encC) > np.abs(CActTH),dtype=int)

### Helper Functions

In [24]:
def getEdges(encodings):
    TH = 0.5
    comb = combinations(np.linspace(0,encodings.shape[1]-1,encodings.shape[1]), 2)
    c = list(comb)
    edges = np.zeros(len(c))  

    for i in range(encodings.shape[0]):
        for n in range(len(c)):
            a = int(c[n][0])
            b = int(c[n][1])
            if (encodings[i][a]>TH) & (encodings[i][b]>TH):
                edges[n] = edges[n]+1
    return edges,c

def learnConcept(examples, name, pattern_complexity,normalize = False):
    e,c = getEdges(examples)
    strongest = e.argsort()[-pattern_complexity:][::-1]
    strongest_combs = []
    strengths = []
    #print('Combinations:      Strength:')
    for s in strongest:
        #print(str(c[s]) + '        ' + str(int(e[s])))
        strongest_combs.append(c[s])
        strengths.append(int(e[s]))
    if normalize:
        strengths = strengths/np.sum(strengths)
    else:
        strengths = np.array(strengths)/examples.shape[0]
    return strongest_combs, strengths


def getActsLogic(neuronCombs, strengths, data, class_TH):
    patternOn = np.zeros(data.shape[0])
    for i,n in enumerate(neuronCombs):
        n1_act = (data[:,int(n[0])] > 0.5)
        n2_act = (data[:,int(n[1])] > 0.5)
        n_act = np.array(n1_act & n2_act, dtype=int) * strengths[i]
        patternOn = patternOn + n_act
    patternOn = (patternOn >= class_TH)
    return patternOn

def plotPatternActsLogic(patternOn, labels):
    
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    plt.plot(patternOn)
    plt.plot(labels/2)
    plt.xlim([0,labels.shape[0]])
    plt.ylim([0,1.1])
    plt.legend(['Pattern On', 'Label Present'])
    plt.yticks([])
    plt.xlabel('Frames')
    
    plt.subplot(1,2,2)
    c = plt.hist(patternOn + labels/2, bins = [0,0.5,1,1.5,2])
    plt.xticks([0.25,0.75,1.25,1.75],['TN', 'FN', 'FP','TP'])
    correct = np.round((c[0][0] + c[0][3])/labels.shape[0]*100,2)
    precision = np.round(c[0][3]/(c[0][3]+c[0][2])*100,2)# If L is predicted it is correct X% of time
    recall = np.round(c[0][3]/(c[0][3]+c[0][1])*100,2)# Correctly identifies X% of L
    plt.title('Correct: ' + str(correct) + '% Precision: ' + str(precision) + '% Recall: ' + str(recall) + '%')
    plt.show()
    
def plotCPR(combs, weights, enc, labels, THs):
    cpr = np.zeros((THs.shape[0],3))
    for i,TH in enumerate(THs):
        patternOn = getActsLogic(combs, weights, enc, class_TH=TH)
        c = plt.hist(patternOn + labels/2, bins = [0,0.5,1,1.5,2])
        
        correct = np.round((c[0][0] + c[0][3])/labels.shape[0]*100,2)
        precision = np.round(c[0][3]/(c[0][3]+c[0][2])*100,2)# If L is predicted it is correct X% of time
        recall = np.round(c[0][3]/(c[0][3]+c[0][1])*100,2)# Correctly identifies X% of L
        
        cpr[i] = np.array([correct,precision,recall])
    plt.clf()
    plt.figure(figsize=(15,5))
    plt.plot(cpr)
    plt.legend(['% Correct', 'Precision', 'Recall'])
    plt.xticks(np.linspace(0,THs.shape[0]-1,THs.shape[0]),np.round(THs,2))
    plt.xlim([0,THs.shape[0]-1])
    plt.ylim([0,100])
    plt.xlabel('Threshold')
    plt.ylabel('Performance [Accuracy in %]')
    plt.show()
    
def getCPR(combs, weights, enc, labels, THs,pattern):
    cpr = np.zeros((THs.shape[0],5))
    for i,TH in enumerate(THs):
        if pattern=='single':
            patternOn = getSingleActsLogic(combs, weights, enc, class_TH=TH)
        elif pattern=='double':
            patternOn = getActsLogic(combs, weights, enc, class_TH=TH)
        elif pattern=='triple':
            patternOn = getTripletsActsLogic(combs, weights, enc, class_TH=TH)
        else:
            print('unknown pattern identifier.')
            break
        c = plt.hist(patternOn + labels/2, bins = [0,0.5,1,1.5,2])
        
        correct = np.round((c[0][0] + c[0][3])/labels.shape[0]*100,2)
        precision = np.round(c[0][3]/(c[0][3]+c[0][2])*100,2)# If L is predicted it is correct X% of time
        recall = np.round(c[0][3]/(c[0][3]+c[0][1])*100,2)# Correctly identifies X% of L
        f1score = np.round(2*(precision*recall/(precision+recall)),2)
        
        cpr[i] = np.array([correct,precision,recall,correct+precision+recall,f1score])
    plt.close()
    return np.nan_to_num(cpr)

def getConceptSubset(AllConceptExamples, numExp):
    subset = np.random.randint(0,AllConceptExamples.shape[0],numExp)
    trainExp = AllConceptExamples[subset]
    mask = np.ones(AllConceptExamples.shape, bool)
    mask[subset] = False
    testExp = AllConceptExamples[mask]
    return trainExp,  testExp

def getFP_FN(o,l):
    w = np.where(l!=o)
    FN = np.sum((o[w]==0)&(l[w]==1))/len(w[0])*100
    FP = np.sum((o[w]>0)&(l[w]==0))/len(w[0])*100
    return np.round(FP,3),np.round(FN,3)

def getAccPR(o,l):
    w = np.where(l!=o)
    trueP = np.sum((o==l)&(o==1))
    trueN = np.sum((o==l)&(o==0))
    falseP = np.sum((o[w]>0)&(l[w]==0))
    falseN = np.sum((o[w]==0)&(l[w]==1))
    Acc = (trueP+trueN)/len(o)*100
    Precision = trueP/(trueP+falseP)*100
    Recall = trueP/(trueP+falseN)*100
    return np.round(Acc,3),np.round(Precision,3),np.round(Recall,3),[trueP,trueN,falseP,falseN]

def printStats(out, labels):
    print('Accuracy: '+str(np.mean(out == labels)*100)+'%')
    print('Everything correct in '+str(np.sum(np.mean(out == labels,axis=1)==1.0)/len(labels)*100)+'%')
    print('Correct Door in '+str(np.round(np.mean(np.array(labels)[:,0]==out[:,0])*100,3))+'%')

    AccND,PND,RND,NDA = getAccPR((out[:,0]==0).astype(int),(labels[:,0]==0).astype(int))
    print('     No Door - Acc: '+str(AccND)+'% Precision: '+str(PND)+'% Recall: '+str(RND)+'%  |  TP,TN,FP,FN: '+str(NDA))

    AccLD,PLD,RLD,LDA = getAccPR((out[:,0]==1).astype(int),(labels[:,0]==1).astype(int))
    print('     Level Door - Acc: '+str(AccLD)+'% Precision: '+str(PLD)+'% Recall: '+str(RLD)+'%  |  TP,TN,FP,FN: '+str(LDA))

    AccGD,PGD,RGD,GDA = getAccPR((out[:,0]==2).astype(int),(labels[:,0]==2).astype(int))
    print('     Green Door - Acc: '+str(AccGD)+'% Precision: '+str(PGD)+'% Recall: '+str(RGD)+'%  |  TP,TN,FP,FN: '+str(GDA))

    AccKD,PKD,RKD,KDA = getAccPR((out[:,0]==3).astype(int),(labels[:,0]==3).astype(int))
    print('     Key Door - Acc: '+str(AccKD)+'% Precision: '+str(PKD)+'% Recall: '+str(RKD)+'%  |  TP,TN,FP,FN: '+str(KDA))

    AccOD,POD,ROD,ODA = getAccPR((out[:,0]==4).astype(int),(labels[:,0]==4).astype(int))
    print('     Other Door - Acc: '+str(AccOD)+'% Precision: '+str(POD)+'% Recall: '+str(ROD)+'%  |  TP,TN,FP,FN: '+str(ODA))

    AccK,PK,RK,KA = getAccPR(out[:,1],np.array(labels)[:,1])
    print('Correct Key in '+str(AccK)+'% - Precision: '+
          str(PK)+'% Recall: '+str(RK)+'%  |  TP,TN,FP,FN: '+str(KA))

    AccO,PO,RO,OA = getAccPR(out[:,2],np.array(labels)[:,2])
    print('Correct Orb in '+str(AccO)+'% - Precision: '+
          str(PO)+'% Recall: '+str(RO)+'%  |  TP,TN,FP,FN: '+str(OA))
    
def getStats(out, labels):

    AccND,PND,RND,NDA = getAccPR((out[:,0]==0).astype(int),(labels[:,0]==0).astype(int))
    
    AccLD,PLD,RLD,LDA = getAccPR((out[:,0]==1).astype(int),(labels[:,0]==1).astype(int))

    AccGD,PGD,RGD,GDA = getAccPR((out[:,0]==2).astype(int),(labels[:,0]==2).astype(int))

    AccKD,PKD,RKD,KDA = getAccPR((out[:,0]==3).astype(int),(labels[:,0]==3).astype(int))

    AccOD,POD,ROD,ODA = getAccPR((out[:,0]==4).astype(int),(labels[:,0]==4).astype(int))

    AccK,PK,RK,KA = getAccPR(out[:,1],np.array(labels)[:,1])

    AccO,PO,RO,OA = getAccPR(out[:,2],np.array(labels)[:,2])
    
    precisions = np.nan_to_num(np.array([PND, PLD, PGD, PKD, POD, PK, PO]))
    recalls = np.nan_to_num(np.array([RND, RLD, RGD, RKD, ROD, RK, RO]))
    f1scores = np.nan_to_num(np.round(2*(precisions*recalls/(precisions+recalls)),2))
    
    stats = {'Accuracy': np.mean(out == labels)*100,
            'Accuracies': np.array([AccND, AccLD, AccGD, AccKD, AccOD, AccK, AccO]),
            'Precisions': precisions,
            'Recalls': recalls,
            'F1Scores': f1scores}
    return stats


### Learning Concepts
#### Standard (Doubles)

In [12]:
def LearnAllConcepts(numExp, pattern_complexity, encoding, to_learn, concept_def):
    allConcepts = []
        
    for i, concept in enumerate(to_learn):
        learnExamples, testExamples = getConceptSubset(np.where(concept_def[i])[0], numExp)

        examples = encoding[learnExamples]
        labels_test = concept_def[i]

        combs, weights = learnConcept(examples, 'no door', pattern_complexity=pattern_complexity,normalize=True)
        
        true_exp = np.where(labels_test==True)[0]
        false_exp = np.where(labels_test==False)[0]
        np.random.shuffle(false_exp)
        balanced_exp = np.append(true_exp,false_exp[:true_exp.shape[0]])

        cpr_train = getCPR(combs, weights, examples, np.ones(examples.shape[0]), np.linspace(0,1,21),pattern='double')
        cpr_test = getCPR(combs, weights, encoding[balanced_exp], labels_test[balanced_exp], np.linspace(0,1,21),pattern='double')

        ConceptStats = {'Concept':concept,
                     'Examples': learnExamples,
                   'TestExamples': testExamples,
                     'ExamplesEnc': examples,
                     'Combinations': combs,
                     'Weights': weights,
                     'CPR_Train': cpr_train,
                     'CPR_Test': cpr_test}

        allConcepts.append(ConceptStats)
    return allConcepts


#### Single Concepts

In [ ]:
def learnSingleConcept(examples, pattern_complexity,normalize = False):
    neuronActs = np.sum(examples,axis=0)
    strongest = neuronActs.argsort()[-pattern_complexity:][::-1]
    strongest_N = []
    strengths = []
    for s in strongest:
        #print(str(s) + '        ' + str(int(neuronActs[s])))
        strongest_N.append(s)
        strengths.append(int(neuronActs[s]))#/examples.shape[0])
    if normalize:
        strengths = strengths/np.sum(strengths)
    else:
        strengths = np.array(strengths)/examples.shape[0]
    return strongest_N, strengths

def getSingleActsLogic(neurons, strengths, data, class_TH):
    #strengths = strengths/np.max(strengths)
    patternOn = np.zeros(data.shape[0])
    for i,n in enumerate(neurons):
        n1_act = (data[:,int(n)] > 0.5)
        n_act = np.array(n1_act, dtype=int) * strengths[i]
        #print(n_act)
        patternOn = patternOn + n_act
    '''print('pattern on: ' + str(patternOn))
    print('Max: ' + str(np.max(patternOn))[:4])
    print('Min: ' + str(np.min(patternOn)))
    print('Mean: ' + str(np.mean(patternOn)))
    print('Median: ' + str(np.median(patternOn)))'''
    #plt.hist(patternOn)
    patternOn = (patternOn >= class_TH)#np.array(neuronCombs).shape[0]-1)
    return patternOn

def LearnAllSingleConcepts(numExp, pattern_complexity, encoding, to_learn, concept_def):
    allConcepts = []
        
    for i, concept in enumerate(to_learn):
        learnExamples, testExamples = getConceptSubset(np.where(concept_def[i])[0], numExp)

        examples = encoding[learnExamples]
        labels_test = concept_def[i]

        combs, weights = learnSingleConcept(examples, pattern_complexity=pattern_complexity,normalize=True)
        
        true_exp = np.where(labels_test==True)[0]
        false_exp = np.where(labels_test==False)[0]
        np.random.shuffle(false_exp)
        balanced_exp = np.append(true_exp,false_exp[:true_exp.shape[0]])

        cpr_train = getCPR(combs, weights, examples, np.ones(examples.shape[0]), np.linspace(0,1,21),pattern='single')
        cpr_test = getCPR(combs, weights, encoding[balanced_exp], labels_test[balanced_exp], np.linspace(0,1,21),pattern='single')
        
        ConceptStats = {'Concept':concept,
                     'Examples': learnExamples,
                   'TestExamples': testExamples,
                     'ExamplesEnc': examples,
                     'Combinations': combs,
                     'Weights': weights,
                     'CPR_Train': cpr_train,
                     'CPR_Test': cpr_test}

        allConcepts.append(ConceptStats)
    return allConcepts

#### Triplet Concepts

In [ ]:
def getTriplets(encodings):
    TH = 0.5
    comb = combinations(np.linspace(0,encodings.shape[1]-1,encodings.shape[1]), 3)
    c = list(comb)
    triplets = np.zeros(len(c))

    for i in range(encodings.shape[0]):
        for n in range(len(c)):
            Na = int(c[n][0])
            Nb = int(c[n][1])
            Nc = int(c[n][2])
            if (encodings[i][Na]>TH) & (encodings[i][Nb]>TH) & (encodings[i][Nc]>TH):
                triplets[n] = triplets[n]+1
    return triplets,c

def learnTripletConcept(examples, pattern_complexity,normalize = False):
    t,c = getTriplets(examples)
    strongest = t.argsort()[-pattern_complexity:][::-1]
    strongest_combs = []
    strengths = []
    #print('Combinations:      Strength:')
    for s in strongest:
        #print(str(c[s]) + '        ' + str(int(t[s])))
        strongest_combs.append(c[s])
        strengths.append(int(t[s]))
    if normalize:
        strengths = strengths/np.sum(strengths)
    else:
        strengths = np.array(strengths)/examples.shape[0]
    return strongest_combs, strengths


def getTripletsActsLogic(neuronCombs, strengths, data, class_TH):
    patternOn = np.zeros(data.shape[0])
    for i,n in enumerate(neuronCombs):
        n1_act = (data[:,int(n[0])] > 0.5)
        n2_act = (data[:,int(n[1])] > 0.5)
        n3_act = (data[:,int(n[2])] > 0.5)
        n_act = np.array(n1_act & n2_act & n3_act, dtype=int) * strengths[i]
        patternOn = patternOn + n_act
    patternOn = (patternOn >= class_TH)
    return patternOn

def LearnAllTripletConcepts(numExp, pattern_complexity, encoding, to_learn, concept_def):
    allConcepts = []
        
    for i, concept in enumerate(to_learn):
        learnExamples, testExamples = getConceptSubset(np.where(concept_def[i])[0], numExp)

        examples = encoding[learnExamples]
        labels_test = concept_def[i]

        combs, weights = learnTripletConcept(examples, pattern_complexity=pattern_complexity,normalize=True)
        
        true_exp = np.where(labels_test==True)[0]
        false_exp = np.where(labels_test==False)[0]
        np.random.shuffle(false_exp)
        balanced_exp = np.append(true_exp,false_exp[:true_exp.shape[0]])

        cpr_train = getCPR(combs, weights, examples, np.ones(examples.shape[0]), np.linspace(0,1,21),pattern='triple')
        cpr_test = getCPR(combs, weights, encoding[balanced_exp], labels_test[balanced_exp], np.linspace(0,1,21),pattern='triple')

        ConceptStats = {'Concept':concept,
                     'Examples': learnExamples,
                     'Combinations': combs,
                     'Weights': weights,
                     'CPR_Train': cpr_train,
                     'CPR_Test': cpr_test}

        allConcepts.append(ConceptStats)
    return allConcepts

#### Support Vector Machine

In [ ]:
def getCPR_SVM(classification, labels, THs, single=False):
    c = plt.hist(classification + labels/2, bins = [0,0.5,1,1.5,2])
        
    correct = np.round((c[0][0] + c[0][3])/labels.shape[0]*100,2)
    precision = np.round(c[0][3]/(c[0][3]+c[0][2])*100,2)# If L is predicted it is correct X% of time
    recall = np.round(c[0][3]/(c[0][3]+c[0][1])*100,2)# Correctly identifies X% of L
    f1score = np.round(2*(precision*recall/(precision+recall)),2)
        
    cpr = np.array([correct,precision,recall,correct+precision+recall,f1score])
    plt.clf()
    return np.nan_to_num(cpr)

def LearnAllConceptsSVM(numExp, encoding, to_learn, concept_def):
    allConcepts = []
        
    for i, concept in enumerate(to_learn):
        learnExamples, testExamples = getConceptSubset(np.where(concept_def[i])[0], numExp)
        learnExamplesNeg, _ = getConceptSubset(np.where(np.logical_not(concept_def[i]))[0], numExp)

        examples = encoding[learnExamples]
        examplesNeg = encoding[learnExamplesNeg]
        labels_test = concept_def[i]

        clf_linear = svm.SVC(kernel='linear')
        clf_linear.fit(np.concatenate([examples,examplesNeg]), np.concatenate([np.ones(examples.shape[0]),np.zeros(examplesNeg.shape[0])]))
        out_linear = clf_linear.predict(encInt)
        
        true_exp = np.where(labels_test==True)[0]
        false_exp = np.where(labels_test==False)[0]
        np.random.shuffle(false_exp)
        balanced_exp = np.append(true_exp,false_exp[:true_exp.shape[0]])

        cpr_test = getCPR_SVM(out_linear[balanced_exp], labels_test[balanced_exp], np.linspace(0,1,21))

        ConceptStats = {'Concept':concept,
                     'Examples': learnExamples,
                     'CPR_Test': cpr_test}

        allConcepts.append(ConceptStats)
    return allConcepts

## FCM
### Define Parameters

In [16]:
to_learn = ['Level Door', 'Green Door', 'Key Door', 'Other Door', 'Key', 'Orb']#'No Door', 
concept_def = [y[:,0]==1, y[:,0]==2, y[:,0]==3, y[:,0]==4, y[:,1]==1, y[:,2]==1]#y[:,0]==0, 
numExp = 5
pattern_complexity = 10

### Test

In [ ]:
allConceptsSingle = LearnAllSingleConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
allConceptsTriplet = LearnAllTripletConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
allConcepts = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
allConceptsSVM = LearnAllConceptsSVM(numExp, encInt, to_learn, concept_def)

### FCM on 4 Network Activations
Comparing FCM on activations of agent, autoencoder, classifier and random.

In [27]:
numRuns = 100
bestTHStatsAECR_TH2050 = pd.DataFrame(columns = {'Concept','Encoding', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

for pattern_complexity in [pattern_complexity]:
    for numExp in [numExp]:
        for i in range(numRuns):
            print(i)
            allConcepts = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsAE = LearnAllConcepts(numExp, pattern_complexity, encAE, to_learn, concept_def)
            allConceptsC = LearnAllConcepts(numExp, pattern_complexity, encC, to_learn, concept_def)
            allConceptsR = LearnAllConcepts(numExp, pattern_complexity, randAct, to_learn, concept_def)
            for c in allConcepts:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Agent',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Agent',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH50[0],
                                                                            'Precision': TH50[1],
                                                                            'Recall': TH50[2],
                                                                            'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
                
            for c in allConceptsAE:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Autoencoder',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Autoencoder',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH50[0],
                                                                            'Precision': TH50[1],
                                                                            'Recall': TH50[2],
                                                                            'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
               
            for c in allConceptsC:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Classifier',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Classifier',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH50[0],
                                                                            'Precision': TH50[1],
                                                                            'Recall': TH50[2],
                                                                            'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
           
            for c in allConceptsR:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Random',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                bestTHStatsAECR_TH2050 = bestTHStatsAECR_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Random',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH50[0],
                                                                            'Precision': TH50[1],
                                                                            'Recall': TH50[2],
                                                                            'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
           


0


c:\users\vkakerbeck\miniconda3\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in double_scalars
c:\users\vkakerbeck\miniconda3\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
bestTHStatsAECR_TH2050.to_csv(figurePath+'bestTHStatsAECRNormx2_TH2050.csv')

In [28]:
bestTHStatsAECR_TH2050.head()

,Precision,TH,Recall,Concept,NumExp,Accuracy,CPR,F1Score,Encoding,Examples,PC
0,78.77,0.2,46.0,Level Door,5,66.8,191.57,58.08,Agent,"[379, 548, 571, 370, 452]",10.0
1,93.10,0.5,32.4,Level Door,5,65.0,190.50,48.07,Agent,"[379, 548, 571, 370, 452]",10.0
2,71.96,0.2,61.6,Green Door,5,68.8,202.36,66.38,Agent,"[602, 605, 638, 622, 686]",10.0
3,83.33,0.5,44.0,Green Door,5,67.6,194.93,57.59,Agent,"[602, 605, 638, 622, 686]",10.0
4,65.09,0.2,27.6,Key Door,5,56.4,149.09,38.76,Agent,"[1008, 1007, 1089, 1071, 871]",10.0


### FCM on all Activations for Different Numbers of Examples

In [ ]:
numRuns = 100
bestTHStatsAECR_numExp = pd.DataFrame(columns = {'Concept','Encoding','PC', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

to_learnD = [to_learn[1]]
concept_defD = [concept_def[1]]
for pattern_complexity in [pattern_complexity]:
    for numExp in [1,2,3,4,5,10,15,20,50]:
        for i in range(numRuns):
            print(i)
            allConcepts = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learnD, concept_defD)
            allConceptsAE = LearnAllConcepts(numExp, pattern_complexity, encAE, to_learnD, concept_defD)
            allConceptsC = LearnAllConcepts(numExp, pattern_complexity, encC, to_learnD, concept_defD)
            allConceptsR = LearnAllConcepts(numExp, pattern_complexity, randAct, to_learnD, concept_defD)
            for c in allConcepts:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][4]
                    bestTHStatsAECR_numExp = bestTHStatsAECR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Agent',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsAE:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][4]
                    bestTHStatsAECR_numExp = bestTHStatsAECR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Autoencoder',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsC:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][4]
                    bestTHStatsAECR_numExp = bestTHStatsAECR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Classifier',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
            for c in allConceptsR:
                if c['Concept']=='Level Door':
                    TH20 = c['CPR_Test'][4]
                    bestTHStatsAECR_numExp = bestTHStatsAECR_numExp.append({'Concept': c['Concept'],
                                                                            'Encoding':'Random',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)


In [ ]:
bestTHStatsAECR_numExp.to_csv(figurePath+'bestTHStatsAECR_numExpLDoor_TH20.csv')

### FCM on all Activations for Different Pattern Complexities

In [ ]:
numRuns = 100
bestTHStatsAECR_PCComp_AllC = pd.DataFrame(columns = {'Concept','Encoding','PC', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

for pattern_complexity in [1,2,3,4,5,10,15,20]:
    for numExp in [numExp]:
        for i in range(numRuns):
            print(str(pattern_complexity) + ' - ' + str(i))
            allConcepts = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsAE = LearnAllConcepts(numExp, pattern_complexity, encAE, to_learn, concept_def)
            allConceptsC = LearnAllConcepts(numExp, pattern_complexity, encC, to_learn, concept_def)
            allConceptsR = LearnAllConcepts(numExp, pattern_complexity, randAct, to_learn, concept_def)
            for c in allConcepts:
                TH20 = c['CPR_Test'][4]
                bestTHStatsAECR_PCComp_AllC = bestTHStatsAECR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Agent',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsAE:
                TH20 = c['CPR_Test'][4]
                bestTHStatsAECR_PCComp_AllC = bestTHStatsAECR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Autoencoder',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
            for c in allConceptsC:
                TH20 = c['CPR_Test'][4]
                bestTHStatsAECR_PCComp_AllC = bestTHStatsAECR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Classifier',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
            for c in allConceptsR:
                TH20 = c['CPR_Test'][4]
                bestTHStatsAECR_PCComp_AllC = bestTHStatsAECR_PCComp_AllC.append({'Concept': c['Concept'],
                                                                            'Encoding':'Random',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
    # intermediate saves in case of memory issues:
    #bestTHStatsAECR_PCComp_AllC.to_csv(figurePath+'bestTHStatsAECR_PCComp_AllC_'+str(pattern_complexity)+'.csv')
    

In [ ]:
bestTHStatsAECR_PCComp_AllC.to_csv(figurePath+'bestTHStatsAECR_PCComp_AllC.csv')

### FCM on all Activations for Different Thresholds

In [ ]:
numRuns = 100
bestTHStatsAECR_TH = pd.DataFrame(columns = {'Concept','Encoding','Pattern Complexity', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall'})

for pattern_complexity in [pattern_complexity]:
    for numExp in [numExp]:
        for i in range(numRuns):
            allConcepts = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsAE = LearnAllConcepts(numExp, pattern_complexity, encAE, to_learn, concept_def)
            allConceptsC = LearnAllConcepts(numExp, pattern_complexity, encC, to_learn, concept_def)
            allConceptsR = LearnAllConcepts(numExp, pattern_complexity, randAct, to_learn, concept_def)
            for c in allConcepts:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAECR_TH = bestTHStatsAECR_TH.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)
                
            for c in allConceptsAE:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAECR_TH = bestTHStatsAECR_TH.append({'Concept': c['Concept'],'Encoding':'Autoencoder','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)
                
            for c in allConceptsC:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAECR_TH = bestTHStatsAECR_TH.append({'Concept': c['Concept'],'Encoding':'Classifier','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)
            for c in allConceptsR:
                for i,THTest in enumerate(c['CPR_Test']):
                    bestTHStatsAECR_TH = bestTHStatsAECR_TH.append({'Concept': c['Concept'],'Encoding':'Random','Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': np.round(np.linspace(0,1,21)[i],2),
                                                  'Accuracy': THTest[0], 'Precision': THTest[1],
                                                  'Recall': THTest[2]},ignore_index=True)


In [ ]:
bestTHStatsAECR_TH.to_csv(figurePath+'bestTHStatsAECRNormx2_TH.csv')

### FCM on Different Agent Reward Types

In [ ]:
numRuns = 100
bestTHStatsA_TH2050 = pd.DataFrame(columns = {'Concept','Encoding','Pattern Complexity', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})
for pattern_complexity in [pattern_complexity]:
    for numExp in [numExp]:
        for i in range(numRuns):
            print(i)
            allConceptsE = LearnAllConcepts(numExp, pattern_complexity, encExt, to_learn, concept_def)
            allConceptsEI = LearnAllConcepts(numExp, pattern_complexity, encExtInt, to_learn, concept_def)
            allConceptsI = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            
            for c in allConceptsE:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsA_TH2050 = bestTHStatsA_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'External',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                bestTHStatsA_TH2050 = bestTHStatsA_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'External',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH50[0],
                                                                            'Precision': TH50[1],
                                                                            'Recall': TH50[2],
                                                                            'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)      
            for c in allConceptsEI:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsA_TH2050 = bestTHStatsA_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Ext+Int',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                bestTHStatsA_TH2050 = bestTHStatsA_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Ext+Int',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH50[0],
                                                                            'Precision': TH50[1],
                                                                            'Recall': TH50[2],
                                                                            'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
            for c in allConceptsI:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsA_TH2050 = bestTHStatsA_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Internal',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                
                bestTHStatsA_TH2050 = bestTHStatsA_TH2050.append({'Concept': c['Concept'],
                                                                            'Encoding':'Internal',
                                                                            'PC':pattern_complexity,
                                                                            'Examples': c['Examples'],
                                                                            'NumExp':numExp,'TH': 0.5,
                                                                            'Accuracy': TH50[0],
                                                                            'Precision': TH50[1],
                                                                            'Recall': TH50[2],
                                                                            'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
                


In [ ]:
bestTHStatsA_TH2050.to_csv(figurePath+'bestTHStatsAgentsNormx2_TH2050.csv')

### FCM with Different Types of Patterns

In [ ]:
numRuns = 100
bestTHStatsA_P = pd.DataFrame(columns = {'Concept','Encoding','Pattern','Pattern Complexity', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

for pattern_complexity in [pattern_complexity]:
    for numExp in [numExp]:
        for i in range(numRuns):
            print(i)
            allConcepts = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsSingle = LearnAllSingleConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            allConceptsTriplet = LearnAllTripletConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
            for c in allConceptsSingle:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsA_P = bestTHStatsA_P.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Single',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                                            'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                bestTHStatsA_P = bestTHStatsA_P.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Single',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': 0.5,
                                                  'Accuracy': TH50[0], 'Precision': TH50[1],
                                                  'Recall': TH50[2],'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
                
            for c in allConcepts:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsA_P = bestTHStatsA_P.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Double',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                                            'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                bestTHStatsA_P = bestTHStatsA_P.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Double',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': 0.5,
                                                  'Accuracy': TH50[0], 'Precision': TH50[1],
                                                  'Recall': TH50[2],'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
            for c in allConceptsTriplet:
                TH20 = c['CPR_Test'][4]
                TH50 = c['CPR_Test'][10]
                bestTHStatsA_P = bestTHStatsA_P.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Triplet',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                                            'TH': 0.2,
                                                                            'Accuracy': TH20[0],
                                                                            'Precision': TH20[1],
                                                                            'Recall': TH20[2],
                                                                            'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
                bestTHStatsA_P = bestTHStatsA_P.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Triplet',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': 0.5,
                                                  'Accuracy': TH50[0], 'Precision': TH50[1],
                                                  'Recall': TH50[2],'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
            

In [ ]:
bestTHStatsA_P.to_csv(figurePath+'bestTHStatsA_Pattern.csv')

### FCM Compared to SVM

In [ ]:
numRuns = 100
bestTHStatsA_SVM = pd.DataFrame(columns = {'Concept','Encoding','Pattern','Pattern Complexity', 'Examples','NumExp','TH', 
                                        'Accuracy', 'Precision', 'Recall','CPR','F1Score'})

for i in range(numRuns):
    print(i)
    allConcepts = LearnAllConcepts(numExp, pattern_complexity, encInt, to_learn, concept_def)
    allConceptsSVM = LearnAllConceptsSVM(numExp, encInt, to_learn, concept_def)
    for c in allConcepts:
        TH20 = c['CPR_Test'][4]
        TH50 = c['CPR_Test'][10]
        bestTHStatsA_SVM = bestTHStatsA_SVM.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Double',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': 0.2,
                                                  'Accuracy': TH20[0], 'Precision': TH20[1],
                                                  'Recall': TH20[2],'CPR':TH20[3],'F1Score':TH20[4]},ignore_index=True)
        bestTHStatsA_SVM = bestTHStatsA_SVM.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'Double',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': 0.5,
                                                  'Accuracy': TH50[0], 'Precision': TH50[1],
                                                  'Recall': TH50[2],'CPR':TH50[3],'F1Score':TH50[4]},ignore_index=True)
    for c in allConceptsSVM:
        bestTHStatsA_SVM = bestTHStatsA_SVM.append({'Concept': c['Concept'],'Encoding':'Agent','Pattern':'SVM',
                                                              'Pattern Complexity':pattern_complexity, 'Examples': c['Examples'],'NumExp':numExp,
                                                  'TH': 0,
                                                  'Accuracy': c['CPR_Test'][0], 'Precision': c['CPR_Test'][1],
                                                  'Recall': c['CPR_Test'][2],'CPR':c['CPR_Test'][3],'F1Score':c['CPR_Test'][4]},ignore_index=True)

In [ ]:
bestTHStatsA_SVM.to_csv(figurePath+'bestTHStatsA_SVM.csv')